In [38]:
import os
import re

path = "src/gui/tabs/machine_parts/base_panel.py"
if os.path.exists(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    
    # 既存の _run メソッドを、エラーを絶対に画面に出す「最強の _run」に差し替える
    match = re.search(r'(?s)(.*?)(    def _run\(self, label: str\) -> None:.*)', content)
    if match:
        header = match.group(1)
        new_run = """    def _run(self, label: str) -> None:
        import traceback
        from gui.utils.code_utils import run_code
        from gui.utils.viewer import _show_viewer
        from IPython.display import display, HTML

        # 画面をリセット
        self.log_out.clear_output()
        self.code_out.clear_output()
        self.viewer_out.clear_output()

        try:
            with self.log_out:
                print(f'▶ {label} のコードを生成中...')
            
            code = self._build_code()
            
            with self.code_out:
                display(HTML(f'<div class="cad-code">{code}</div>'))
            
            with self.log_out:
                print(f'▶ {label} を実行中...')
            
            ok, err, obj = run_code(code)
            
            with self.log_out:
                self.log_out.clear_output()
                if ok:
                    print('✅ 生成成功！ビューアーに送信します。')
                    _show_viewer(obj, self.viewer_out)
                else:
                    print(f'❌ 【実行エラー】:\\n{err}')
                    
        except Exception as e:
            with self.log_out:
                self.log_out.clear_output()
                print(f'💥 【システムエラー（強制キャッチ）】:\\n{traceback.format_exc()}')
"""
        with open(path, "w", encoding="utf-8") as f:
            f.write(header + new_run)
        print("✅ 実行ログの画面出力を完全に繋ぎ直しました！")
    else:
        print("⚠️ _run メソッドの差し替えに失敗しました。")
else:
    print("⚠️ base_panel.py が見つかりません。")

✅ 実行ログの画面出力を完全に繋ぎ直しました！


In [57]:
from build123d import *
from ocp_vscode import show

show(Box(10, 10, 10))

c


In [58]:
print("1. 処理を開始します...")

from build123d import *
from ocp_vscode import show, set_port

print("2. ライブラリを読み込みました。")

# ポートを3939に固定
set_port(3939)

with BuildPart() as test_part:
    Box(10, 10, 10)

print("3. キューブのデータを送信します！")
show(test_part)

print("✅ すべて完了しました！")

1. 処理を開始します...
2. ライブラリを読み込みました。
3. キューブのデータを送信します！
c
✅ すべて完了しました！


In [59]:
import sys
import os

# --- 1. 環境の自動判定 ---
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("🌐 Colab環境を検出しました。Colab用のセットアップを開始します...")
    
    # Colab用ウィジェットの許可
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except ImportError:
        pass

    # Colab用のクリーンインストール（UIを破壊しない安全な構成）
    !pip uninstall -y websockets google-genai ocp-vscode ocp-cad-viewer jupyter-cadquery
    !pip install -q build123d anthropic openai google-genai ipywidgets

    # リポジトリの展開
    BRANCH = "06"
    REPO_URL = "https://github.com/WOCae/Build123d.git"
    REPO_NAME = "Build123d"
    
    %cd /content
    if not os.path.exists(REPO_NAME):
        !git clone -b {BRANCH} {REPO_URL}
    
    %cd /content/{REPO_NAME}

    # Colab専用ビューアー（絶対にUIが消えない Three.js 方式）
    viewer_code = """
import os
import base64
import ipywidgets as w
from IPython.display import display, HTML

def _show_viewer(obj, out_widget):
    out_widget.clear_output()
    if obj is None: return
    try:
        target = obj.part if hasattr(obj, 'part') else obj
        tmp_stl = "/content/tmp_view.stl"
        from build123d import export_stl
        export_stl(target, tmp_stl)
        with open(tmp_stl, 'rb') as f:
            stl_b64 = base64.b64encode(f.read()).decode()
        uid = f'v{abs(hash(tmp_stl)) % 99999:05d}'
        html = f'''
        <div id="wrap_{uid}" style="width:100%; height:450px; background:#1e1e1e; border-radius:8px; border:1px solid #444;"></div>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
        <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/controls/OrbitControls.js"></script>
        <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/loaders/STLLoader.js"></script>
        <script>
        (function(){{
            var container = document.getElementById('wrap_{uid}');
            var scene = new THREE.Scene();
            var camera = new THREE.PerspectiveCamera(45, container.clientWidth / container.clientHeight, 0.1, 1000);
            camera.position.set(70, 70, 70);
            var renderer = new THREE.WebGLRenderer({{antialias: true, alpha: true}});
            renderer.setSize(container.clientWidth, container.clientHeight);
            container.appendChild(renderer.domElement);
            var controls = new THREE.OrbitControls(camera, renderer.domElement);
            controls.enableDamping = true;
            scene.add(new THREE.AmbientLight(0xffffff, 0.6));
            var dirLight = new THREE.DirectionalLight(0xffffff, 0.8);
            dirLight.position.set(100, 100, 50);
            scene.add(dirLight);
            var binaryString = window.atob('{stl_b64}');
            var len = binaryString.length;
            var bytes = new Uint8Array(len);
            for (var i = 0; i < len; i++) {{ bytes[i] = binaryString.charCodeAt(i); }}
            var geometry = new THREE.STLLoader().parse(bytes.buffer);
            geometry.computeVertexNormals();
            geometry.computeBoundingBox();
            var center = geometry.boundingBox.getCenter(new THREE.Vector3());
            var mesh = new THREE.Mesh(geometry, new THREE.MeshStandardMaterial({{color: 0x4a90d9, metalness: 0.3, roughness: 0.4}}));
            mesh.position.sub(center);
            scene.add(mesh);
            function animate() {{ requestAnimationFrame(animate); controls.update(); renderer.render(scene, camera); }}
            animate();
        }})();
        </script>
        '''
        with out_widget: display(HTML(html))
    except Exception as e:
        with out_widget: print(f"⚠️ 3D表示エラー: {e}")

def _find_latest_stl(code): return None
"""

else:
    print("💻 ローカル環境を検出しました。VS Code用のセットアップを開始します...")
    
    # ローカルの場合は pip install や git clone を一切行わない（環境破壊を防ぐため）
    
    # ローカル専用ビューアー（VS Code のサイドパネル OCP CAD Viewer に送信）
    viewer_code = """
import os
import ipywidgets as w
from IPython.display import display

def _show_viewer(obj, out_widget):
    out_widget.clear_output()
    if obj is None:
        with out_widget:
            print("⚠️ 表示するオブジェクトがありません。")
        return
    try:
        target = obj.part if hasattr(obj, 'part') else obj
        # VS Code の OCP CAD Viewer にモデルを送信
        from ocp_vscode import show
        show(target)
        with out_widget:
            print("✅ VS Code の「OCP CAD Viewer」タブにモデルを送信しました！")
            print("👉 画面右側（または別タブ）のビューアーを確認してください。")
    except Exception as e:
        with out_widget:
            print(f"⚠️ ocp_vscode 送信エラー: {e}")
            print("💡 VS Code の OCP CAD Viewer 拡張機能が起動しているか確認してください。")

def _find_latest_stl(code): return None
"""

# --- 2. viewer.py の書き換えとパスの設定 ---
viewer_path = "src/gui/utils/viewer.py"
os.makedirs(os.path.dirname(viewer_path), exist_ok=True)
with open(viewer_path, "w", encoding="utf-8") as f:
    f.write(viewer_code)

src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"✅ セットアップ完了！ ({'Colabモード' if IS_COLAB else 'ローカル/VS Codeモード'})")

💻 ローカル環境を検出しました。VS Code用のセットアップを開始します...
✅ セットアップ完了！ (ローカル/VS Codeモード)


In [ ]:
import build123d_gui
build123d_gui.display_dashboard()

✅ GUI 起動完了  ─  まず「⚙️ API設定」タブでモードとキーを設定してください


# Build123d 基本動作確認ノートブック

このノートブックでは、Build123dのインストールと基本的な3Dモデル生成を確認します。

**流れ**
1. インストール
2. 基本形状の生成（Box, Cylinder, Sphere）
3. ブーリアン演算（足し算・引き算）
4. フィレット・面取り
5. 機械部品の例（フランジ付きシャフト）
6. ファイル出力（STEP / STL）
7. LLMプロンプトテンプレート

---
> **LLMを使って3Dモデルを生成したい場合は、セル7のプロンプトをコピーして使ってください。**

## 1. インストール

In [61]:
# build123dのインストール（初回のみ）
import sys
%pip install -q build123d

# Colabの場合、インストール後にランタイム再起動が必要
try:
    from google.colab import runtime
    print('✅ build123d インストール完了')
    print('⚠️  ランタイムを再起動してください（ランタイム → ランタイムを再起動）')
    print('   再起動後はこのセル(セル1)をスキップして、セル2から実行してください。')
except ImportError:
    print('✅ build123d インストール完了（ローカル環境）')


Note: you may need to restart the kernel to use updated packages.
✅ build123d インストール完了（ローカル環境）


In [62]:
# インポートと動作確認
from build123d import *
import os

print("✅ build123d インポート成功")
print(f"   バージョン確認: Box(10,10,10) → {Box(10,10,10)}")

✅ build123d インポート成功
   バージョン確認: Box(10,10,10) → Box at 0x1b3712803d0, label(), #children(0)


## 2. 基本形状の生成

In [63]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 3. ブーリアン演算（足し算・引き算）

In [64]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [65]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [66]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 4. フィレット・面取り

In [67]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 5. 機械部品の例：フランジ付きシャフト

LLMが生成するコードの典型的な例です。

In [68]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [69]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 6. ファイル出力（STEP / STL）

- **STEP**: FreeCADやSolidWorksで開ける汎用形式
- **STL**: 3Dプリントや後続のメッシャーへ渡す形式

In [70]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [71]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## (参考) 旧プロンプトテンプレート

APIキーをお持ちでない場合、セル7-Bの `LLM_MODE = 'manual'` を使うと  
同等のプロンプトが自動生成されます。このセクションは参考情報です。


In [72]:
def show_prompt_template():
    PROMPT_TEMPLATE = """
あなたはBuild123dのエキスパートです。以下のルールに従ってPythonコードを生成してください。

【ルール】
1. 必ず `from build123d import *` でインポートする
2. 形状は `with BuildPart() as part:` のコンテキストマネージャ内に書く
3. 寸法はすべてmm単位のfloatまたはintで書く
4. パラメータは変数として冒頭にまとめて定義する（変更しやすくするため）
5. ブーリアン演算は mode=Mode.SUBTRACT（穴・引き算）または mode=Mode.ADD（合体）を使う
6. スケッチで穴を作る場合は BuildSketch をネストせず、以下のように mode=Mode.SUBTRACT を使う：
   with BuildSketch() as sk:
       Circle(outer_r)
       Circle(inner_r, mode=Mode.SUBTRACT)
7. フィレットは `fillet(part.edges(), radius=R)` で指定する
8. 最後に以下の2行でファイル出力する：
   export_step(part.part, "output.step")
   export_stl(part.part, "output.stl")
9. コード以外の説明文は不要。コードブロックのみ返す。

【基本形状リファレンス】
- Box(length, width, height)
- Cylinder(radius=R, height=H)
- Sphere(radius=R)
- Cone(bottom_radius=R1, top_radius=R2, height=H)

【スケッチ押し出しリファレンス】
- 断面形状を作ってから押し出す方法（パイプ・複雑断面に使う）:
  with BuildSketch() as sk:
      Circle(outer_r)                        # 外形
      Circle(inner_r, mode=Mode.SUBTRACT)    # 内穴（BuildSketchのネストは使わない）
  extrude(amount=length)

【配置・パターン】
- Locations((x, y, z)): 指定座標に移動
- PolarLocations(radius, count): 円周上に均等配置
- GridLocations(x_spacing, y_spacing, x_count, y_count): グリッド配置

【よくある間違い・注意事項】
- BuildSketch を入れ子（ネスト）にしない
- 穴はすべて mode=Mode.SUBTRACT で指定する
- fillet はすべての形状が確定した後、最後に適用する

【作りたいもの】
"""
    print("📋 以下をLLMにコピー＆ペーストして、最後に作りたいものを追記してください：")
    print("="*60)
    print(PROMPT_TEMPLATE)
    print("="*60)
    print("\n例：【作りたいもの】")
    print("外径60mm、内径40mm、長さ100mmのパイプ。両端にR3のフィレットあり。")

## 8. 手動貼り付け実行セル（manualモード用）

`LLM_MODE = 'manual'` でLLMから返ってきたコードをここに貼り付けて実行してください。  
`LLM_MODE = 'api'` の場合はセル7-Bで自動実行されるため、このセルは不要です。


In [73]:
# ─────────────────────────────────────────────────────────────
# LLMコードの手動実行は 📋 Manual タブに移動しました。
# GUIの「📋 Manual」タブでコードを貼り付けて実行してください。
# ─────────────────────────────────────────────────────────────


---
## 補足メモ

**Colabでの3Dプレビュー**  
Colab上でのインタラクティブな3D表示は現時点では制限があります。  
STLファイルを出力して、別途Three.jsビューワーや外部ツール（MeshLab、FreeCAD等）で確認してください。

**ローカル環境でのプレビュー**  
VS Code + `ocp_vscode` 拡張機能を使うとリアルタイムで3D確認できます。
```
pip install ocp_vscode
```

**後続パーツとの連携**  
- STLファイル → pygmshメッシャーへ渡す  
- STEPファイル → FreeCADやCalculiXのCAEツールへ渡す

**参考リンク**  
- [Build123d 公式ドキュメント](https://build123d.readthedocs.io/)  
- [Build123d GitHub](https://github.com/gumyr/build123d)

In [74]:
# Manualタブでプロンプトテンプレートを表示したい場合は、
# show_prompt_template() を実行してください。
# 例: show_prompt_template()